<a href="https://colab.research.google.com/github/bessabra/PRO2PRO/blob/main/DadosMixSei.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mixResultadoGeral = pd.concat([baserq, basedf]).drop_duplicates(subset=['SIAPE', 'SIAPE'], keep=False)

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

basedf = pd.read_excel('/content/Dados Funcionais v01.01.xlsx')
baserq = pd.read_excel('/content/LISTA_SEI_PROFESSORES_CONSOLIDADAv27.12.xlsx')

In [ ]:
# baserq = baserq.rename(
#     columns = {'IT-NO-SERVIDOR':'NOME'}
#     )

basedf['INATIVIDADE'] = pd.to_datetime(basedf['INATIVIDADE'], errors="coerce")
basedf['OBITO']       = pd.to_datetime(basedf['OBITO'], errors="coerce")

baserq['DATA DO REQUERIMENTO'] = pd.to_datetime(baserq['DATA DO REQUERIMENTO'], errors="coerce")

In [ ]:
# Junta as planilhas basedf e baserq pra identificar correspondência de CPF, SIAPE e NOME
mixResultadoGeral = baserq.merge(
    basedf, on=['SIAPE'], 
    how='outer', 
    # how='right', 
    suffixes=['', '_'], 
    indicator=True
    )

# Adiciona uma coluna com legendas para cada situação
mixResultadoGeral.loc[(mixResultadoGeral['_merge'] == 'right_only'), 'CONDIÇÃO']  = 'Servidor não fez requerimento'
mixResultadoGeral.loc[(mixResultadoGeral['_merge'] == 'left_only'), 'CONDIÇÃO']   = 'Servidor não encontrado em Dados Funcionais'
mixResultadoGeral.loc[(mixResultadoGeral['_merge'] == 'both'), 'CONDIÇÃO']        = 'Servidor encontrado em Dados Funcionais'


mixResultadoGeral.to_excel('ResultadoGeralMixv01.01.xlsx')

# Cria uma nova planilha somente com os servidores não encontrados na Dados Funcionais v13.12
servNao = mixResultadoGeral.loc[(mixResultadoGeral['_merge'] == 'left_only')]
servNao.to_excel('servNaoEncv09.01.xlsx')


In [ ]:
# Cria uma nova planilha somente com os servidores com requerimento e que foram encontrados na Dados Funcionais v13.12
servEncontrados = basedf.merge(
    baserq, on=['SIAPE'], 
    how='inner', 
    suffixes=['', '_'], 
    indicator=True
    )

servEncontrados.loc[(servEncontrados['_merge'] == 'both'), 'CONDIÇÃO'] = 'Servidor encontrado em Dados Funcionais'

servEncontrados.to_excel('servEncontradosv09.01.xlsx')